# Problem 1: Training a Simple Chatbot using a Seq-to-Seq Model (25 points)

You will train a simple chatbot using movie scripts from the Cornell Movie Dialogs Corpus based on the [PyTorch Chatbot Tutorial](https://pytorch.org/tutorials/beginner/chatbot_tutorial.html).

This tutorial allows you to train a recurrent sequence-to-sequence model. You will learn the following concepts:

- Handle loading and pre-processing of [the Cornell Movie-Dialogs Corpus dataset](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)
- Implement a sequence-to-sequence model with [Luong attention mechanism(s)](https://arxiv.org/abs/1508.04025)
- Jointly train encoder and decoder models using mini-batches
- Implement greedy-search decoding module
- Interact with the trained chatbot

---

## Scoring Breakdown

| Task | Points |
|------|--------|
| Task 1: Run the tutorial end-to-end in Colab | 5 |
| Task 3: Create W&B sweep configuration | 5 |
| Task 4: Run hyperparameter sweeps on GPU Colab | 5 |
| Task 5: Analysis of best hyperparameters & feature importance | 10 |
| **Total** | **25** |

---

## References
- [The Cornell Movie Dialogs Corpus](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)
- [Hyperparameter sweeps with Weights and Biases (video tutorial)](https://www.youtube.com/watch?v=9zrmUIlScdY)
- [Sample Google Colab project for W&B sweeps](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb)
- [Weights and Biases Website](https://wandb.ai/site)

---
## Task 1 [5 points]: Run the Tutorial End-to-End

Make a copy of the [PyTorch Chatbot Tutorial](https://pytorch.org/tutorials/beginner/chatbot_tutorial.html) notebook, follow the instructions to train and evaluate the chatbot model in your **Google Colab** environment (GPU recommended).

The tutorial code is provided below as your starting point. Run each cell in order and verify that the model trains successfully and you can interact with the chatbot at the end.

### Setup: Install Dependencies and Download Data

The Cornell Movie-Dialogs Corpus must be downloaded before running the tutorial. The dataset is available via [ConvoKit](https://convokit.cornell.edu/documentation/movie.html).

In [1]:
# Install dependencies (torch is pre-installed in Colab; run the %%writefile cell above first)
!pip install "torch>=2.4.0" -q
!pip install "convokit>=3.0,<4.0" -q
!pip install "wandb>=0.18" -q

# Download the Cornell Movie-Dialogs Corpus via ConvoKit
import convokit
corpus = convokit.Corpus(filename=convokit.download("movie-corpus"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 18.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.2 MB/s eta 0:00:00
No configuration file found at /root/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
model_directory: ~/.convokit/saved-models
default_backend: mem


In [ ]:
# Install required packages (run once in Colab)
# Requires PyTorch >= 2.4.0 (pre-installed in Colab; verify with: import torch; print(torch.__version__))
!pip install "convokit>=3.0,<4.0" -q
!pip install "wandb>=0.18" -q

# Download the Cornell Movie-Dialogs Corpus via ConvoKit
import convokit
corpus = convokit.Corpus(filename=convokit.download("movie-corpus"))

### Preparations

In [2]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json

assert torch.__version__ >= "2.4", f"PyTorch >= 2.4 required, got {torch.__version__}"

if torch.accelerator.is_available():
    device = torch.accelerator.current_accelerator().type
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
print(f"Using {device} device")

Using cuda device


### Load & Preprocess Data

The Cornell Movie-Dialogs Corpus is stored in `utterances.jsonl` format. We parse the raw file to extract consecutive question-answer sentence pairs from each conversation.

In [3]:
def loadLinesAndConversations(fileName):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            lineJson = json.loads(line)
            lineObj = {}
            lineObj["lineID"] = lineJson["id"]
            lineObj["characterID"] = lineJson["speaker"]
            lineObj["text"] = lineJson["text"]
            lines[lineObj['lineID']] = lineObj

            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj

    return lines, conversations


def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        for i in range(len(conversation["lines"]) - 1):
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

#### Create Formatted Data File

Parse the corpus and write tab-separated input/output pairs to a text file for training.

In [4]:
# Define paths — update corpus_path to where ConvoKit downloaded the dataset
corpus_name = "movie-corpus"
corpus_path = os.path.join("/root/.convokit/saved-corpora", corpus_name)
datafile = os.path.join(corpus_path, "formatted_movie_lines.txt")
save_dir = os.path.join("data", "save")

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nProcessing corpus...")
lines, conversations = loadLinesAndConversations(
    os.path.join(corpus_path, "utterances.jsonl")
)

print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

print("\nSample lines from file:")
with open(datafile, 'rb') as f:
    lines_sample = f.readlines()
for line in lines_sample[:3]:
    print(line)


Processing corpus...

Writing newly formatted file...

Sample lines from file:
b'They do to!\tThey do not!\n'
b'She okay?\tI hope so.\n'
b"Wow\tLet's go.\n"


#### Vocabulary Class

The `Voc` class maintains word-to-index and index-to-word mappings. Three special tokens are reserved:
- `PAD` (0): padding token used to equalize batch sequence lengths
- `SOS` (1): start-of-sequence token fed as the first decoder input
- `EOS` (2): end-of-sequence token appended to every target sequence

In [5]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3
        for word in keep_words:
            self.addWord(word)

#### Text Normalization & Data Loading

In [6]:
MAX_LENGTH = 10  # Maximum sentence length (in words) to consider

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([\[.!?\]])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

def readVocs(datafile, corpus_name):
    print("Reading lines...")
    lines = open(datafile, encoding='utf-8').read().strip().split('\n')
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

voc, pairs = loadPrepareData(corpus_name, corpus_name, datafile, save_dir)
print("\nSample pairs:")
for pair in pairs[:5]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64313 sentence pairs
Counting words...
Counted words: 18082

Sample pairs:
['they do to !', 'they do not !']
['she okay ?', 'i hope so .']
['wow', 'let s go .']
['what good stuff ?', 'the real you .']
['the real you .', 'like my fear of wearing pastels ?']


#### Trim Rare Words

Remove words appearing fewer than `MIN_COUNT` times to reduce vocabulary size and improve generalization.

In [7]:
MIN_COUNT = 3

def trimRareWords(voc, pairs, MIN_COUNT):
    voc.trim(MIN_COUNT)
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break
        if keep_input and keep_output:
            keep_pairs.append(pair)
    print("Trimmed from {} pairs to {}, {:.4f} of total".format(
        len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)
    ))
    return keep_pairs

pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7833 / 18079 = 0.4333
Trimmed from 64313 pairs to 53131, 0.8261 of total


### Prepare Data for Models

Convert sentence pairs into padded tensors suitable for batch training. Sequences in a batch are padded to the same length, and a binary mask is created so that the loss function ignores padding positions.

In [8]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

def inputVar(l, voc):
    """Returns padded input sequence tensor and lengths."""
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

def outputVar(l, voc):
    """Returns padded target sequence tensor, mask, and max target length."""
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

def batch2TrainData(voc, pair_batch):
    """Returns all items for a given batch of pairs."""
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len

# Sanity check
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches
print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[   7, 5319,   11,  547, 4607],
        [ 161,   92,   44,    6,   14],
        [  19, 1162,    5,  547,    2],
        [   3,   72,   18,    6,    0],
        [ 713,   25,   14,    2,    0],
        [  72,   36,    2,    0,    0],
        [  14,   14,    0,    0,    0],
        [  14,    2,    0,    0,    0],
        [  14,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([10,  8,  6,  5,  3])
target_variable: tensor([[6567,   19,  109,  829,   24],
        [   7,   17,   24,   85,   64],
        [ 490,   79,  898,    6,  511],
        [ 109,  917,   72,    2,  321],
        [  24,  578,   34,    0, 1413],
        [  10,   10,   10,    0,   14],
        [   2,    2,    2,    0,    2]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True, False,  True],
        [ True, 

### Define Models

#### Encoder

The encoder is a bidirectional GRU. For each input token it produces a hidden state; the forward and backward outputs are **summed** to form a single context vector per time step.

In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding
        # TODO: Define a bidirectional GRU with the given hidden_size, n_layers, and dropout.
        # Use nn.GRU — remember to set bidirectional=True.
        emb_dim = embedding.embedding_dim

        # NOTE: In PyTorch GRU, dropout is only applied if n_layers > 1
        self.gru = nn.GRU(
            input_size=emb_dim,
            hidden_size=hidden_size,
            num_layers=n_layers,
            dropout=(0 if n_layers == 1 else dropout),
            bidirectional=True,
        )

    def forward(self, input_seq, input_lengths, hidden=None):
        # TODO: Embed input_seq, pack the padded sequence, run through the GRU,
        # unpack, then SUM the forward and backward outputs to get a single
        # context vector per time step. Return (outputs, hidden).
        # 1) Embed tokens -> (max_len, batch, emb_dim)
        embedded = self.embedding(input_seq)

        # 2) Pack (assumes sequences are sorted by length desc; your batch2TrainData does that)
        packed = torch.nn.utils.rnn.pack_padded_sequence(
            embedded,
            input_lengths.cpu(),   # safer for pack on some setups
            enforce_sorted=True
        )

        # 3) Run through BiGRU
        packed_outputs, hidden = self.gru(packed, hidden)

        # 4) Unpack -> outputs: (max_len, batch, hidden_size*2)
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_outputs)

        # 5) Sum bidirectional outputs: (max_len, batch, hidden_size)
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:]

        # 6) Sum bidirectional hidden states too:
        # hidden originally: (n_layers*2, batch, hidden_size)
        # reshape split directions: forward layers [0:n_layers], backward layers [n_layers:2*n_layers]
        hidden = hidden[:self.n_layers] + hidden[self.n_layers:]

        return outputs, hidden


#### Attention Layer

The [Luong attention mechanism](https://arxiv.org/abs/1508.04025) computes a context vector as a weighted sum of encoder outputs. Three scoring functions are supported:

| Method | Formula |
|--------|---------|
| `dot` | $h_t^\top \bar{h}_s$ |
| `general` | $h_t^\top W_a \bar{h}_s$ |
| `concat` | $v_a^\top \tanh(W_a [h_t ; \bar{h}_s])$ |

In [10]:
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        # TODO: For 'general', create a Linear(hidden_size, hidden_size).
        # For 'concat', create Linear(hidden_size*2, hidden_size) and a learnable
        # parameter vector v of size hidden_size.
        # For "general": score(h_t, h_s) = h_t^T W_a h_s
        if self.method == 'general':
            self.attn = nn.Linear(hidden_size, hidden_size)

        # For "concat": score(h_t, h_s) = v_a^T tanh(W_a [h_t ; h_s])
        elif self.method == 'concat':
            self.attn = nn.Linear(hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        # TODO: Compute element-wise product and sum over the last dimension.
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        # TODO: Apply self.attn to encoder_output, then dot with hidden.
        energy = self.attn(encoder_output)              # (src_len, batch, hidden)
        return torch.sum(hidden * energy, dim=2)        # (src_len, batch)

    def concat_score(self, hidden, encoder_output):
        # TODO: Expand hidden to match encoder_output shape, concatenate,
        # apply self.attn + tanh, then dot with self.v.
        src_len = encoder_output.size(0)

        # Expand hidden from (1, batch, hidden) -> (src_len, batch, hidden)
        hidden_expanded = hidden.expand(src_len, -1, -1)

        # Concat on last dim -> (src_len, batch, 2*hidden)
        concat_input = torch.cat((hidden_expanded, encoder_output), dim=2)

        # Apply linear + tanh -> (src_len, batch, hidden)
        energy = torch.tanh(self.attn(concat_input))

        # Dot with v: (hidden,) with (src_len, batch, hidden) -> (src_len, batch)
        # Use broadcasting: energy * v, then sum over hidden
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # TODO: Dispatch to the correct scoring function based on self.method,
        # transpose the energies, and return a softmax probability distribution
        # with an added dimension (shape: batch x 1 x src_len).
        # 1) Compute attention energies: (src_len, batch)
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        else:  # 'dot'
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # 2) Transpose to (batch, src_len)
        attn_energies = attn_energies.t()

        # 3) Softmax over src_len, then unsqueeze -> (batch, 1, src_len)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)


#### Decoder

The `LuongAttnDecoderRNN` generates one output token per step. It attends to encoder outputs via the `Attn` module, concatenates the context vector with the GRU output, and projects the result to a vocabulary-sized distribution.

In [11]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        # TODO: Define a unidirectional GRU (hidden_size -> hidden_size, n_layers).
        emb_dim = embedding.embedding_dim
        self.gru = nn.GRU(
            input_size=emb_dim,
            hidden_size=hidden_size,
            num_layers=n_layers,
            dropout=(0 if n_layers == 1 else dropout),
            bidirectional=False
        )
        # TODO: Define a concat Linear(hidden_size*2, hidden_size) to merge context + GRU output.
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        # TODO: Define an output Linear(hidden_size, output_size) for the vocabulary projection.
        self.out = nn.Linear(hidden_size, output_size)
        # TODO: Instantiate an Attn(attn_model, hidden_size) attention module.
        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # TODO:
        # 1. Embed input_step and apply dropout.
        # embedded: (1, batch, emb_dim)
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # 2. Run through self.gru to get rnn_output and new hidden state.
        # rnn_output: (1, batch, hidden_size)
        # hidden:     (n_layers, batch, hidden_size)
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # 3. Compute attention weights over encoder_outputs using self.attn.
        # attn_weights: (batch, 1, src_len)
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # 4. Compute context vector via batch matrix multiply (bmm).
        # encoder_outputs -> (batch, src_len, hidden_size)
        encoder_outputs = encoder_outputs.transpose(0, 1)
        # context: (batch, 1, hidden_size)
        context = attn_weights.bmm(encoder_outputs)
        # 5. Concatenate rnn_output and context, apply self.concat + tanh.
        # rnn_output: (batch, 1, hidden_size)
        rnn_output = rnn_output.transpose(0, 1)
        # concat_input: (batch, 1, 2*hidden_size)
        concat_input = torch.cat((rnn_output, context), dim=2)
        # concat_output: (batch, hidden_size)
        concat_output = torch.tanh(self.concat(concat_input)).squeeze(1)
        # 6. Project to vocabulary size with self.out and apply softmax.
        # output: (batch, output_size)
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)

        return output, hidden


### Define Training Procedure

#### Masked NLL Loss

Because sequences are padded to the same length within a batch, we compute loss only over non-padding positions using a binary mask.

In [12]:
def maskNLLLoss(inp, target, mask):
    """Compute NLL loss over non-padded positions only.

    Args:
        inp:    (batch, vocab_size) softmax probabilities from the decoder
        target: (batch,) ground-truth token indices
        mask:   (batch,) boolean mask — True for real tokens, False for PAD
    Returns:
        loss (scalar tensor), nTotal (int count of real tokens)
    """
    # TODO:
    # 1. Count the number of non-padded tokens (mask.sum()).
    nTotal = mask.sum()
    # 2. Gather the log-probability of the correct token for each item in the batch.
    gathered = torch.gather(inp, 1, target.unsqueeze(1)).squeeze(1)
    # 3. Select only the masked (real) tokens and take the mean.
    loss = -torch.log(gathered + 1e-12)
    # 4. Move loss to device and return (loss, nTotal).
    loss = loss.masked_select(mask).mean()
    # Ensure tensor is on correct device
    loss = loss.to(inp.device)
    return loss, nTotal.item()

#### Single Training Iteration

The `train` function performs one forward and backward pass over a single batch. Key techniques:

- **Teacher forcing**: with probability `teacher_forcing_ratio`, the ground-truth token is fed as the next decoder input instead of the model's own prediction. Higher values accelerate early convergence but may hurt generalization.
- **Gradient clipping**: gradients are clipped to a maximum norm of `clip` to prevent exploding gradients, which are common in RNN training.

In [25]:
def train(input_variable, lengths, target_variable, mask, max_target_len,
          encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer,
          batch_size, clip, max_length=MAX_LENGTH):
    """Run one mini-batch forward + backward pass.

    Steps:
      1. Zero gradients on both optimizers.
      2. Move tensors to device (keep lengths on CPU for pack_padded_sequence).
      3. Run encoder to get encoder_outputs and encoder_hidden.
      4. Initialize decoder_input with SOS tokens (shape: 1 x batch_size).
      5. Set decoder_hidden from encoder_hidden[:decoder.n_layers].
      6. Decide teacher forcing: if random() < teacher_forcing_ratio use ground truth,
         otherwise use the decoder's own top-1 prediction as the next input.
      7. Loop over max_target_len steps, accumulate maskNLLLoss.
      8. loss.backward(), clip gradients for both encoder and decoder, step optimizers.
    Returns:
      Average loss per real token (float).
    """
    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Move tensors to device (lengths stay on CPU)
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Encoder forward pass
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Initialize decoder input with SOS tokens
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Initialize decoder hidden state from encoder
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Decide teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    loss = 0
    print_losses = []
    n_totals = 0

    # Decode one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )

            # Teacher forcing: next input is ground truth
            decoder_input = target_variable[t].view(1, -1)

            mask_loss, nTotal = maskNLLLoss(
                decoder_output,
                target_variable[t],
                mask[t]
            )

            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )

            # No teacher forcing: use model prediction
            _, topi = decoder_output.topk(1)
            decoder_input = topi.transpose(0, 1).detach()

            mask_loss, nTotal = maskNLLLoss(
                decoder_output,
                target_variable[t],
                mask[t]
            )

            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Backpropagation
    loss.backward()

    # Gradient clipping
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Optimizer step
    encoder_optimizer.step()
    decoder_optimizer.step()

    # Return average loss per real token
    return sum(print_losses) / n_totals

#### Training Loop

`trainIters` manages the full training loop: printing average loss every `print_every` iterations and saving checkpoints every `save_every` iterations.

In [14]:
def trainIters(model_name, voc, pairs, encoder, decoder,
               encoder_optimizer, decoder_optimizer, embedding,
               encoder_n_layers, decoder_n_layers, save_dir,
               n_iteration, batch_size, print_every, save_every,
               clip, corpus_name, loadFilename):

    training_batches = [
        batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
        for _ in range(n_iteration)
    ]

    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        loss = train(input_variable, lengths, target_variable, mask, max_target_len,
                     encoder, decoder, embedding,
                     encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(
                iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        if iteration % save_every == 0:
            directory = os.path.join(
                save_dir, model_name, corpus_name,
                '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size)
            )
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

### Define Evaluation

#### Greedy Search Decoder

At inference time we use greedy decoding: at each step, select the token with the highest probability and feed it as input to the next step.

In [18]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        """Greedily decode up to max_length tokens.

        Steps:
          1. Run self.encoder to get encoder_outputs and encoder_hidden.
          2. Initialize decoder_hidden and decoder_input (SOS token).
          3. At each step: run self.decoder, take the argmax token,
             append it to all_tokens and its score to all_scores.
          4. Feed the chosen token back as the next decoder input.
        Returns:
          (all_tokens, all_scores) — both 1-D tensors of length max_length.
        """
        # 1) Encoder forward
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)

        # 2) Prepare decoder initial hidden state (match decoder layers)
        decoder_hidden = encoder_hidden[:self.decoder.n_layers]

        # 3) Initialize decoder input with SOS
        decoder_input = torch.LongTensor([[SOS_token]]).to(input_seq.device)

        all_tokens = torch.zeros([0], dtype=torch.long, device=input_seq.device)
        all_scores = torch.zeros([0], dtype=torch.float, device=input_seq.device)

        # 4) Greedy decode
        for _ in range(max_length):
            decoder_output, decoder_hidden = self.decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )

            # decoder_output: (batch=1, vocab)
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # decoder_input: (1,) -> shape to (1,1) for next step
            decoder_input = decoder_input.view(1, 1)

            all_tokens = torch.cat((all_tokens, decoder_input.squeeze(0)), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)

        return all_tokens, all_scores


def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    """Convert a normalized sentence string to a list of decoded word strings."""
    # TODO:
    # 1. Convert sentence to index tensor (indexesFromSentence), get lengths.
    indexes_batch = [indexesFromSentence(voc, sentence)]
    lengths = torch.tensor([len(indexes_batch[0])])
    # 2. Transpose to (seq_len, 1) for the encoder.
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # 3. Run searcher to get token indices.
    input_batch = input_batch.to(device)
    # 4. Map indices back to words via voc.index2word.
    tokens, scores = searcher(input_batch, lengths, max_length)
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    """Interactive loop: read input from stdin, print bot response. Type 'q' to quit."""
    input_sentence = ''
    while True:
        try:
            input_sentence = input('> ')
            if input_sentence == 'q' or input_sentence == 'quit':
                break
            input_sentence = normalizeString(input_sentence)
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))
        except KeyError:
            print("Error: Encountered unknown word.")


### Model Initialization & Run Training

Configure the model hyperparameters, build the encoder and decoder, initialize optimizers, and start training.

In [16]:
# ---- Model hyperparameters ----
model_name = 'cb_model'
attn_model = 'dot'        # attention scoring: 'dot', 'general', or 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# ---- Training hyperparameters ----
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 100
save_every = 500

loadFilename = None  # set to a .tar checkpoint path to resume training

# ---- Build models ----
embedding = nn.Embedding(voc.num_words, hidden_size)
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size,
                               voc.num_words, decoder_n_layers, dropout)
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

# ---- Optimizers ----
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(),
                               lr=learning_rate * decoder_learning_ratio)

# ---- Train ----
encoder.train()
decoder.train()
trainIters(model_name, voc, pairs, encoder, decoder,
           encoder_optimizer, decoder_optimizer, embedding,
           encoder_n_layers, decoder_n_layers, save_dir,
           n_iteration, batch_size, print_every, save_every,
           clip, corpus_name, loadFilename)

Models built and ready to go!
Initializing ...
Training...
Iteration: 100; Percent complete: 2.5%; Average loss: 5.0112
Iteration: 200; Percent complete: 5.0%; Average loss: 4.2540
Iteration: 300; Percent complete: 7.5%; Average loss: 3.9635
Iteration: 400; Percent complete: 10.0%; Average loss: 3.8652
Iteration: 500; Percent complete: 12.5%; Average loss: 3.7658
Iteration: 600; Percent complete: 15.0%; Average loss: 3.6954
Iteration: 700; Percent complete: 17.5%; Average loss: 3.6515
Iteration: 800; Percent complete: 20.0%; Average loss: 3.5729
Iteration: 900; Percent complete: 22.5%; Average loss: 3.5492
Iteration: 1000; Percent complete: 25.0%; Average loss: 3.5084
Iteration: 1100; Percent complete: 27.5%; Average loss: 3.4693
Iteration: 1200; Percent complete: 30.0%; Average loss: 3.4304
Iteration: 1300; Percent complete: 32.5%; Average loss: 3.3885
Iteration: 1400; Percent complete: 35.0%; Average loss: 3.3457
Iteration: 1500; Percent complete: 37.5%; Average loss: 3.3157
Iteratio

### Interact with the Chatbot

Switch models to evaluation mode and start a conversation. Type `q` to quit.

In [19]:
encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)
evaluateInput(encoder, decoder, searcher, voc)

> hello
Bot: hello . . . . .
> what are you
Bot: i m sorry . . . .
> why sorry
Bot: why ? my heart is mine .
> that is deep
Bot: yes . . . . .
> why so deep?
Bot: i don t know . . .
> are you confused?
Bot: yes . . . . .
> why are you confused
Bot: i m not a good idea . .
> no you are a good idea
Bot: no i m not
> why are you not
Bot: why not ? you re dead . .
> what the fuck?
Bot: what ? the bay . . .
> I am not dead
Bot: you re not ? you re dead .
> I AM NOT DEAD
Bot: you re not ? you re dead .
> FUCK OFF
Bot: you re a girl . . .
> q


---
## Task 2: Learn Weights & Biases (W&B) — No Points, Required for Tasks 3–5

Before proceeding, watch the [Hyperparameter Sweeps with W&B video tutorial](https://www.youtube.com/watch?v=9zrmUIlScdY) and review the [accompanying Colab notebook](https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb).

Then install and authenticate the W&B library below.

In [20]:
# wandb was already installed with a version pin in the setup cell above
import wandb
wandb.login()  # Enter your API key when prompted — sign up free at https://wandb.ai/site

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanishpatel01 (tanishpatel01-columbia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

---
## Task 3 [5 points]: Create a W&B Sweep Configuration

Define a sweep configuration using the **W&B Random Search** strategy over the following hyperparameters:

| Hyperparameter | Search values |
|---|---|
| `learning_rate` | 0.0001, 0.00025, 0.0005, 0.001 |
| `optimizer` | adam, sgd |
| `clip` | 0, 25, 50, 100 |
| `teacher_forcing_ratio` | 0, 0.5, 1.0 |
| `decoder_learning_ratio` | 1.0, 3.0, 5.0, 10.0 |

The sweep should **minimize** the metric `train_loss`.

**Hints:**
- Use `method: "random"` and specify each hyperparameter under `parameters` with a `values` list.
- Instrument your training loop with `wandb.init(config=...)` and `wandb.log({"train_loss": ...})`.
- Register your sweep with `sweep_id = wandb.sweep(sweep_config, project="chatbot-sweep")`.

In [21]:
# TODO: Define sweep_config using W&B random search over the hyperparameters listed above.
# Then register the sweep:
#   sweep_id = wandb.sweep(sweep_config, project='chatbot-sweep')
sweep_config = {
    "method": "random",
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"values": [1e-4, 2.5e-4, 5e-4, 1e-3]},
        "optimizer": {"values": ["adam", "sgd"]},
        "clip": {"values": [0, 25, 50, 100]},
        "teacher_forcing_ratio": {"values": [0.0, 0.5, 1.0]},
        "decoder_learning_ratio": {"values": [1.0, 3.0, 5.0, 10.0]},
    },
}

sweep_id = wandb.sweep(sweep_config, project="chatbot-sweep")
print("Sweep ID:", sweep_id)


Create sweep with ID: fny2xfoy
Sweep URL: https://wandb.ai/tanishpatel01-columbia/chatbot-sweep/sweeps/fny2xfoy
Sweep ID: fny2xfoy


### Instrument the Training Loop for W&B

Write a `train_sweep()` function that:
1. Calls `wandb.init()` to start a new run
2. Reads hyperparameter values from `wandb.config` (e.g. `wandb.config.learning_rate`)
3. Builds the models and optimizers using those values
4. Calls your training loop and logs `train_loss` at each `print_every` step with `wandb.log()`

In [26]:
def train_sweep():
    """Single sweep run invoked by the W&B agent.

    Steps:
      1. Call wandb.init() (use as a context manager).
      2. Read hyperparameters from wandb.config
         (learning_rate, optimizer, clip, teacher_forcing_ratio, decoder_learning_ratio).
      3. Build fresh encoder/decoder and optimizers using those values.
      4. Run the training loop; after every print_every iterations call
         wandb.log({'train_loss': avg_loss, 'iteration': iteration}).
    """
    with wandb.init() as run:

        config = wandb.config

        # 1️⃣ Read hyperparameters
        learning_rate = config.learning_rate
        optimizer_name = config.optimizer
        clip = config.clip
        teacher_forcing_ratio = config.teacher_forcing_ratio
        decoder_learning_ratio = config.decoder_learning_ratio

        # Make teacher_forcing_ratio global (used inside train())
        teacher_forcing_ratio = teacher_forcing_ratio

        # 2️⃣ Build fresh models
        hidden_size = 500
        encoder_n_layers = 2
        decoder_n_layers = 2
        dropout = 0.1

        embedding = nn.Embedding(voc.num_words, hidden_size)

        encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout).to(device)
        decoder = LuongAttnDecoderRNN(
            attn_model="dot",
            embedding=embedding,
            hidden_size=hidden_size,
            output_size=voc.num_words,
            n_layers=decoder_n_layers,
            dropout=dropout
        ).to(device)

        # 3️⃣ Optimizers
        if optimizer_name.lower() == "adam":
            encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
            decoder_optimizer = optim.Adam(
                decoder.parameters(),
                lr=learning_rate * decoder_learning_ratio
            )
        else:  # SGD
            encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
            decoder_optimizer = optim.SGD(
                decoder.parameters(),
                lr=learning_rate * decoder_learning_ratio
            )

        # 4️⃣ Training setup
        n_iteration = 4000
        batch_size = 64
        print_every = 100

        print_loss = 0

        training_batches = [
            batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
            for _ in range(n_iteration)
        ]

        # 5️⃣ Training loop
        for iteration in range(1, n_iteration + 1):

            training_batch = training_batches[iteration - 1]
            input_variable, lengths, target_variable, mask, max_target_len = training_batch

            loss = train(
                input_variable, lengths,
                target_variable, mask, max_target_len,
                encoder, decoder, embedding,
                encoder_optimizer, decoder_optimizer,
                batch_size, clip
            )

            print_loss += loss

            if iteration % print_every == 0:
                avg_loss = print_loss / print_every
                print_loss = 0

                wandb.log({
                    "train_loss": avg_loss,
                    "iteration": iteration
                })

---
## Task 4 [5 points]: Run Hyperparameter Sweeps on GPU Colab

Launch a W&B sweep agent on a **GPU-enabled** Colab runtime (Runtime > Change runtime type > T4 GPU). The agent will automatically sample configurations and execute `train_sweep()` for each one.

- Run at least **10 sweep trials** to cover a meaningful portion of the search space.
- Monitor results live in the [W&B console](https://wandb.ai).
- Paste your W&B project URL or a screenshot of the sweep results dashboard below.

In [ ]:
# TODO: Launch the W&B sweep agent.
# Run at least 10 trials on a GPU-enabled Colab runtime.
#   wandb.agent(sweep_id, function=train_sweep, count=10)
# YOUR CODE HERE


**W&B Project Link / Screenshot:**

*Paste your W&B sweep URL here (e.g. https://wandb.ai/\<username\>/chatbot-sweep/sweeps/\<sweep-id\>) or embed a screenshot of the parallel coordinates / loss curves.*

---
## Task 5 [10 points]: Analysis of Best Hyperparameters & Feature Importance

After completing your sweeps, answer all four questions below.

### 5a. Best Configuration
Report the hyperparameter values that achieved the **lowest `train_loss`** across all sweep runs.

In [ ]:
# (Optional) Retrieve the best run programmatically via the W&B API:
#
# api = wandb.Api()
# runs = api.runs("<your-entity>/chatbot-sweep")
# best_run = min(runs, key=lambda r: r.summary.get("train_loss", float("inf")))
# print("Best config:", best_run.config)
# print("Best loss: ", best_run.summary["train_loss"])

**Best hyperparameter configuration:**

| Hyperparameter | Best Value |
|---|---|
| `learning_rate` | *(your answer)* |
| `optimizer` | *(your answer)* |
| `clip` | *(your answer)* |
| `teacher_forcing_ratio` | *(your answer)* |
| `decoder_learning_ratio` | *(your answer)* |
| **Best `train_loss`** | *(your answer)* |

### 5b. Feature Importance

Use the **W&B feature importance panel** (available in the sweep UI under "Parameter Importance") to identify which hyperparameters had the greatest and least impact on `train_loss`.

*Paste a screenshot of the feature importance chart here and list the top-3 most important hyperparameters.*

### 5c. Convergence Analysis

Explain, in your own words, **why** the top hyperparameters from 5b affect model convergence. Address each of the following:

- **Learning rate** — how does its magnitude affect gradient update steps and the risk of overshooting minima?
- **Optimizer choice (Adam vs SGD)** — how do adaptive vs. fixed learning rates influence training on sparse/noisy sequence data?
- **Gradient clipping (`clip`)** — why does clipping stabilize RNN training, and what happens when `clip=0` (no clipping) or `clip=100` (very loose)?
- **Teacher forcing ratio** — how does the tradeoff between training with ground-truth vs. predicted tokens affect convergence speed and exposure bias?
- **Decoder learning ratio** — why might the decoder benefit from a different learning rate than the encoder?

*Write your analysis here (aim for 200–400 words).*

### 5d. Chatbot Quality

Load the best checkpoint and interact with the chatbot. Report **at least 5 example exchanges** and briefly comment on the quality of the responses.

```
> <your input>
Bot: <model output>

> <your input>
Bot: <model output>
```

*Replace the template above with your actual exchanges.*